In [50]:
import pandas as pd
# import seaborn as sns

In [51]:
# file = input("which file?: ")
# file = "Data/optimization (3).csv"
file = "Data/optimization.csv"
matches = pd.read_csv(file)

# constants
NUM_PARAMS = len(matches.loc[0]) // 2 - 1
WIN_PERC_PRECISION = 1

decisive_matches = matches.query("cros1 != cros2")
tied_matches = matches.query("cros1 == cros2")

# count the wins and losses
league = decisive_matches.groupby(list(matches.columns[:NUM_PARAMS])).size().reset_index(name='wins')
league["losses"] = decisive_matches.groupby(list(matches.columns[NUM_PARAMS:NUM_PARAMS*2])).size().reset_index(name='losses')["losses"]

# count the ties
league["ties"] = 0
tied_league = tied_matches.groupby(list(matches.columns[:NUM_PARAMS])).size().reset_index(name='ties')
for i in range(len(tied_league)):
    league.loc[pd.DataFrame(league[list(league.columns[:NUM_PARAMS])]==list(tied_league.loc[i][:NUM_PARAMS])).all(axis=1), "ties"] = tied_league["ties"][i]

In [52]:
# compute the win percentage and rank the players by it
league["matches"] = league["losses"] + league["wins"] + league["ties"]
league["win_perc"] = round(league["wins"] / league["matches"] * 100, WIN_PERC_PRECISION)
league = league.sort_values("win_perc",ascending=False).reset_index(drop=True)

league.iloc[:,:]

,stra,perc,circ,samp,vert,wins,losses,ties,matches,win_perc
0,Annealing,0.85,0.475,25,1,1,1,0,2,50.0
1,Annealing,0.90,0.475,25,1,2,2,0,4,50.0


In [53]:
# sns.heatmap(league.corr(numeric_only=True), cmap="vlag", center=0.0)

In [54]:
def show_matches_of_rank(rank,which="both"):
    wins = matches[pd.DataFrame(matches[list(matches.columns[:NUM_PARAMS])]==list(league.loc[rank][:NUM_PARAMS])).all(axis=1)].reset_index(drop=True)
    losses = matches[pd.DataFrame(matches[list(matches.columns[NUM_PARAMS:NUM_PARAMS*2])]==list(league.loc[rank][:NUM_PARAMS])).all(axis=1)].reset_index(drop=True)
    both = pd.concat([wins,losses]).reset_index(drop=True)
    return wins if which == "wins" else losses if which == "losses" else both

In [55]:
RANK = 0
show_matches_of_rank(RANK)

,stra,perc,circ,samp,vert,stra2,perc2,circ2,samp2,vert2,cros1,cros2
0,Annealing,0.85,0.475,25,1,Mirroring,0.00,0.475,25,0,4336,2966
1,Annealing,0.90,0.475,25,1,Annealing,0.85,0.475,25,1,97,94
